In [1]:
import matplotlib
matplotlib.use('Agg')

%load_ext autoreload
%autoreload 2

%matplotlib tk
%autosave 180

import sys
sys.path.append("/home/cat/code/widefield/") # go to parent dir



import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import numpy as np
import os

import glob2

from sklearn import linear_model
from scipy import stats
import numpy as np
import matplotlib.patches as mpatches


# visualize results module
from visualize import visualize_svm as Visualize

# 
#data_dir = '/media/cat/4TBSSD/yuki/'
data_dir = '/media/cat/4TBSSD/yuki/'


Autosaving every 180 seconds


In [2]:
##########################################################################
##########################################################################
##########################################################################

def compute_significance(data,
                         significance):

        #print ("self.data: ", data.shape)
        
        mean = data.mean(1)
        #
        sig = []
        for k in range(data.shape[0]):
            #res = stats.ks_2samp(self.data[k],
            #                     control)
            #res = stats.ttest_ind(first, second, axis=0, equal_var=True)

            #
            res = scipy.stats.ttest_1samp(data[k], 0.5)

            sig.append(res[1])


        sig_save = np.array(sig).copy()
        #print ("Self sig save: ", sig_save.shape)

        # multiple hypothesis test
        temp = np.array(sig)
        #print ("data into multi-hypothesis tes:", temp.shape)
        temp2 = multipletests(temp, alpha=significance, method='fdr_bh')
        sig = temp2[1]

        #
        sig=np.array(sig)[None]

        #
        thresh = significance
        idx = np.where(sig>thresh)
        sig[idx] = np.nan

        #
        idx = np.where(mean<0.5)
        sig[:,idx] = np.nan
        #print ("SIG: ", sig.shape)

        # find earliest 
        earliest_continuous = 0
        for k in range(sig.shape[1]-1,0,-1):
            if sig[0][k]<=significance:
                earliest_continuous = k
            else:
                break
                
        earliest_continuous = -(sig.shape[1]-earliest_continuous)/30.
                
        return sig, earliest_continuous
    
    
class LinearRegression(linear_model.LinearRegression):
    """
    LinearRegression class after sklearn's, but calculate t-statistics
    and p-values for model coefficients (betas).
    Additional attributes available after .fit()
    are `t` and `p` which are of the shape (y.shape[1], X.shape[1])
    which is (n_features, n_coefs)
    This class sets the intercept to 0 by default, since usually we include it
    in X.
    """

    def __init__(self, *args, **kwargs):
        if not "fit_intercept" in kwargs:
            kwargs['fit_intercept'] = False
        super(LinearRegression, self)\
                .__init__(*args, **kwargs)

    def fit(self, X, y, n_jobs=1):
        self = super(LinearRegression, self).fit(X, y, n_jobs)

        sse = np.sum((self.predict(X) - y) ** 2, axis=0) / float(X.shape[0] - X.shape[1])
        se = np.array([
            np.sqrt(np.diagonal(sse[i] * np.linalg.inv(np.dot(X.T, X))))
                                                    for i in range(sse.shape[0])
                    ])

        self.t = self.coef_ / se
        self.p = 2 * (1 - stats.t.cdf(np.abs(self.t), y.shape[0] - X.shape[1]))
        return self
    
 

def plot_edt_distributions():

    fig = plt.figure(figsize=(20,12))
    legend_size = 22
    for ctr, animal_id in enumerate(vis.animal_ids):
        ax=plt.subplot(2,3,ctr+1)
        fnames = []
        fnames.append('/media/cat/4TBSSD/yuki/'+animal_id+'/first_decoding_time.npz')
        fnames.append('/media/cat/4TBSSD/yuki/'+animal_id+'/first_decoding_time_lockout.npz')
        fnames.append('/media/cat/4TBSSD/yuki/'+animal_id+'/first_decoding_time_concatenated.npz')

        # 
        shifts=[+15,+15,0]
        linestyles = ['-','-','-']
        titles = ['all','lockout','concatenated']
        pvals = [0.05,0.01,0.001,0.0001,0.00001]
        edts = []
        yvals = []
        for ctr_type, fname in enumerate(fnames):
            #print (" ", animal_id, fname)
            data = np.load(fname, allow_pickle=True)

            # 
            clr = vis.colors[ctr_type]
            res = vis.plot_first_decoding_time_curves(data,
                                                        titles[ctr_type],
                                                        shifts[ctr_type],
                                                        linestyles[ctr_type],
                                                        clr)
            edts.append(res[0])
            yvals.append(res[1])

        if ctr==0:
            plt.legend(fontsize=legend_size)

        temp = np.hstack(yvals)
        ymax = np.max(temp)
        #print ("ymax", ymax)
        #
        plt.xlim(-15,0)
        plt.ylim(bottom=0)    
        plt.xticks([])
        plt.yticks([])
        plt.plot([-3,-3],[0,ymax*1.1],'--',linewidth=5,c='grey')
        plt.plot([-10,-10],[0,ymax*1.1],'--',linewidth=5,c='grey')
        #
        #ax2=
        ax2 = ax.twinx()  # instantiate a second axes that shares the same x-axis

        patches = []
        res = stats.ks_2samp(edts[0], edts[1])
        print (res)
        label_ = ''
        for k in range(len(pvals)):
            if res[1]<pvals[k]:
                label_ = label_ + "*"
            else:
                break

        patches.append(mpatches.Patch(color='blue', label=label_))

        res = stats.ks_2samp(edts[0], edts[2])
        label_ = ''
        for k in range(len(pvals)):
            if res[1]<pvals[k]:
                label_ = label_ + "*"
            else:
                break

        patches.append(mpatches.Patch(color='red', label=label_))

        #print ('')
        plt.legend(handles=patches,
                   fontsize=legend_size)



        plt.xlim(-15,0)
        plt.ylim(bottom=0)
        plt.xticks([])
        plt.yticks([])

    ############################################################
    ############################################################
    ############################################################
    if False:
        plt.show()
    else:
        plt.savefig('/home/cat/first.svg',dpi=1200)
        plt.close()
        
     

In [3]:
##########################
####### INITIALIZE #######
##########################
# LEVER PULL
vis = Visualize.Visualize()

# lever-related data
vis.main_dir = data_dir
vis.random_flag = False  # shuffle data to show baseline

# 
vis.window = 15
vis.lockout_window = 10
vis.code = 'left_paw'
vis.lockout = False

vis.pca_var = 0.95
vis.pca_flag = True


In [21]:
##########################################################
######## PLOT SINGLE SESSION SVM DECODING SESSION ########
##########################################################
vis.significance = 0.05
vis.linewidth=10

#
vis.cbar_offset = 0
vis.smooth_window = 30
vis.lockout = False
vis.xvalidation = 10
vis.sliding_window = 30


vis.imaging_rate = 30

# 
vis.min_trials = 10

############################
# Fig 2E
vis.window = 30

# examples > 10 sec
#vis.animal_id = "AQ2"
#vis.session_id = 'Feb12_'
#vis.session_id = 'Mar15_'
#vis.session_id = 'Mar18_'
#vis.session_id = 'Apr4_'
#vis.session_id = 'Apr22_'


vis.cbar_thick = 0.05

# # 


# # 
# vis.save_fname = '/home/cat/predict.png'

alpha = 1.0
#sessions = ['IJ2pm_Feb29_30Hz','IJ2pm_Mar1_30Hz','IJ2pm_Mar2_30Hz','IJ2pm_Mar3_30Hz','IJ2am_Mar7_30Hz' ]
#session = 'IJ2pm_Mar3_30Hz'
#vis.animal_id = 'IJ2'
#session = 'AQ2am_Feb4_30Hz'
#vis.animal_id='AQ2'
session = 'IA1pm_Feb3_30Hz'
vis.animal_id='IA1'

# Fig 1 used: 
#   session = 'IA1pm_Feb3_30Hz'
#   vis.animal_id='IA1'

#   session = 'IJ2pm_Mar3_30Hz'
#   vis.animal_id = 'IJ2'
#

vis.fname = None

# vis.code = '_limb, layer 1 - left'  # uses a single ROI to decode
vis.code = 'code_04'                  # uses all ROIs/PCA to decode
vis.pickle = False
plot_single = True

vis.ideal_window_flag = False
vis.dlc_offset_flag = False
vis.edt_offset = 0

vis.fig=plt.figure(figsize=(18,10))
vis.ax = plt.subplot(111)

vis.compute_roi=False
vis.shift = 0

if plot_single:
    print ("SESSION: ", session)
    fname_out = 'single'
    vis.n_trials_plotting=[]
    vis.shift_SVM = True
    vis.n_trials=200
    vis.session_id = session
    vis.shift_flag = False
    vis.apply_lever_offset = False   # this is for sihfting DLC triggered data
    vis.xlim = [-20,0]
    vis.show_EDT = True
    vis.edt_offset_y = .1
    vis.edt_offset_x = 1.5#
    vis.show_title = False
    vis.show_legend = False
    vis.legend_location = 1
    vis.cbar=False

    # 
    vis.cmap = "Reds_r"
    vis.alpha = alpha
    vis.plot_significant('red',vis.session_id)

else:
    fname_out = 'concatenated'

    vis.n_trials=200
    vis.session=session
    vis.session='concatenated'
    vis.session_id = 'concatenated'
    vis.fname = '/media/cat/4TBSSD/yuki/IJ2/SVM_Scores/SVM_Scores_concatenated_code_04_trial_ROItimeCourses_30sec_Xvalid10_Slidewindow30.npz'
    vis.cmap = "Reds_r"
    vis.xlim = [-15,0]
    vis.title = '20pca'
    vis.edt_offset_y = .1
    vis.edt_offset_x = 3
    vis.cbar = False
    vis.title_offset = -0.05
    vis.plot_significant_concatenated('red',vis.session_id)

    vis.cbar_offset=0

if True:
    plt.savefig('/home/cat/'+session+'.svg')
    plt.close()
else:
    plt.suptitle(session+" lever pull decoding")
    plt.show()


SESSION:  IA1pm_Feb3_30Hz
DATA;  (1771, 10)
... DLC shift applied for lever pull:  0
 post trials data size  (1771, 10)
      n-trials:  62
self n trials:  62
vmin, vmax;  0.0 0.05 ticks:  [0.         0.01666667 0.03333333 0.05      ]


In [65]:
def compute_shift(main_dir, animal_id, session):

    fname_correlate = os.path.join(main_dir, animal_id,
                                     'tif_files', session,
                                     'correlate.npz')

    try:
        data = np.load(fname_correlate,allow_pickle=True)
    except:
        print( " ... data missing correlate.npz", self.fname)
        data = np.zeros((0))
        return

    cors = data['cors'].squeeze().T

    shift_id_str = '1'
    #print ("SELF SHIFT ID: ", shift_id_str)
    if len(shift_id_str)>1:
        shift_id = int(shift_id_str[0])
        shift_additional = float(shift_id_str[1:])
    else:
        shift_id = int(shift_id_str)
        shift_additional = 0

    print ( " using shift: ", shift_id+shift_additional)

    corr_featur_id = shift_id

    temp_trace = cors[:,corr_featur_id]
    temp_trace[:2000] = 0
    temp_trace[-2000:] = 0
    shift = round(np.argmax(temp_trace)/1000. - 15.,2)+shift_additional
    print ("SHIFT Loaded: ", shift)
    
    return shift


data = np.load('/media/cat/4TBSSD/yuki/AQ2/SVM_Scores/SVM_Scores_ROI_AQ2am_Feb9_30Hz_limb, layer 1 - left_trial_ROItimeCourses_30sec_Xvalid10_Slidewindow30.npz',
              allow_pickle=True)
d = data['accuracy']
print (d.shape)

data = np.load('/media/cat/4TBSSD/yuki/IA1/tif_files/IA1pm_Feb3_30Hz/IA1pm_Feb3_30Hz_jaw_random_ROItimeCourses_15sec_pca_0.95.npy')
print (data.shape)

fig=plt.figure()
plt.plot(data.mean(0).T)

data = np.load('/media/cat/4TBSSD/yuki/IA1/tif_files/IA1pm_Feb3_30Hz/IA1pm_Feb3_30Hz_jaw_trial_ROItimeCourses_15sec_pca_0.95.npy')
print (data.shape)

main_dir = '/media/cat/4TBSSD/yuki/'
animal_id = 'IA1'
session = 'IA1pm_Feb3_30Hz'
shift = compute_shift(main_dir, animal_id, session)

blue = np.load('/media/cat/4TBSSD/yuki/IA1/tif_files/IA1pm_Feb3_30Hz/blue_light_frame_triggers.npz',
              allow_pickle=True)

start = blue['start_blue']

print (start)

total_shift = int(shift*30)-start
print ("Final shift: ", total_shift)

# 
print (data.shape)
plt.plot(np.roll(data.mean(0),total_shift,axis=1).T+5)
plt.plot([450,450],[0,10],'--')
plt.show()

(1771, 10)
(43, 7, 901)
(43, 7, 901)
 using shift:  1
SHIFT Loaded:  5.18
61
Final shift:  94
(43, 7, 901)


In [152]:
def load_data(fname_whole_stack,
             fname_lever_times,
             fname_blue_light):

    # this is the entire session PCA stack
    d1 = np.load(fname_whole_stack)
    
    # zero out beginning and end, usually they are screwed up by excitation light
    d1[:200]=d1[200]
    d1[-200:]=d1[-200]

    # these are the times in seconds for each of the rewarded lever pulls
    imaging_rate = 30
    times = np.loadtxt(fname_lever_times)
    times = np.int32(times*imaging_rate)
    print ("trigger times in frame times: ", times)
    
    # this is the calcium excitation light, we need to shift the dataset to match it
    blue = np.load(fname_blue_light)
    start = blue['start_blue']
    d1 = np.roll(d1,-start,axis=0)
    
    # make a time triggered stack to visualize there is indeed a peak in the correct place
    stack = []
    for t in times:
        temp = d1[t-450:t+450]
        if len(temp)==900:
            stack.append(temp)

    
    stack = np.array(stack)

    return stack, d1, times

#########################################
fname_whole_stack = '/media/cat/4TBSSD/yuki/IA1/tif_files/IA1pm_Feb3_30Hz/IA1pm_Feb3_30Hz_whole_stack_trial_ROItimeCourses_15sec_pca30components.npy'
fname_lever_times = '/media/cat/4TBSSD/yuki/IA1/tif_files/IA1pm_Feb3_30Hz/IA1pm_Feb3_30Hz_all_locs_selected.txt'
# fname_lever_times = '/media/cat/4TBSSD/yuki/IA1/tif_files/IA1pm_Feb3_30Hz/IA1pm_Feb3_30Hz_all_locs_selected_unrewarded.txt'
fname_blue_light = '/media/cat/4TBSSD/yuki/IA1/tif_files/IA1pm_Feb3_30Hz/blue_light_frame_triggers.npz'

# 
stack, data_pca, trigger_times = load_data(fname_whole_stack,
                                         fname_lever_times,
                                         fname_blue_light)

#
print ("PCA data shape: ", data_pca.shape)

# get trial average
d3 = stack.mean(0)

# subtract mean to bring it to 0
d3 -= d3.mean(0)

# plot first 3 pcs
ax=plt.subplot(2,1,1)
t=np.arange(data_pca.shape[0])/30.
plt.plot(t[500:6500],data_pca[500:6500,0]-data_pca[:,0].mean(0), label='pc1')
plt.plot(t[500:6500],data_pca[500:6500,1]-data_pca[:,1].mean(0), label='pc2')
plt.title("PCA of continuous data (1000 frames)")
plt.scatter(trigger_times/30., np.zeros(trigger_times.shape[0]),
           s=200,c='red', label='Trigger times')

plt.xlim(0,200)
plt.legend()

# 
ax=plt.subplot(2,1,2)
t=np.arange(d3.shape[0])/30.-15
plt.plot(t,d3[:,0].T, label='pc1')
plt.plot(t,d3[:,1].T, label='pc2')
plt.plot(t,d3[:,2].T, label='pc3')

# plot some extra bits
plt.plot([t[450],t[450]],
         [-10000,10000],'--',label='t=0sec')
plt.plot([t[0],t[899]],[
    0,0],'--')
plt.title("PCA of trial average data")
plt.legend()
plt.xlim(t[0],t[899])
plt.xlabel("time (sec)")
plt.show()

trigger times in frame times:  [ 2159  4088  5834  7047  7366  7611  7815  8026  8383  8745 11519 12415
 13136 13765 14060 14427 15531 16462 16716 16891 17379 18876 19053 19742
 20620 20940 21311 21597 21813 22407 22695 24105 24492 24719 25043 25424
 26489 26687 27118 27527 28351 28582 28944 29345 30055 30347 31194 31422
 32253 32503 32740 32922 33416 33602 33924 34777 35042 35427 36475 36644
 37273 37540]
PCA data shape:  (40000, 30)


In [113]:

# make sure locs
locs_44threshold = np.load('/media/cat/4TBSSD/yuki/IA1/tif_files/IA1pm_Feb3_30Hz/IA1pm_Feb3_30Hz_locs44threshold.npy')

#print ("Locs 44 threshold: ", locs_44threshold)
codes = np.load('/media/cat/4TBSSD/yuki/IA1/tif_files/IA1pm_Feb3_30Hz/IA1pm_Feb3_30Hz_code44threshold.npy')
code = b'04'
code = b'02'
idx = np.where(codes==code)[0]
locs_selected_unrewarded = locs_44threshold[idx]

print (locs_selected_unrewarded)
np.savetxt('/media/cat/4TBSSD/yuki/IA1/tif_files/IA1pm_Feb3_30Hz/IA1pm_Feb3_30Hz_all_locs_selected_unrewarded.txt',
       locs_selected_unrewarded)

[  77.82628989   78.20717096   81.38418388   85.04007292  140.955724
  204.39744782  207.5378859   236.91444302  237.83537483  239.47154403
  239.69595385  240.78585887  248.74043584  272.63850904  274.05663896
  274.90261602  275.7130549   280.29589796  284.69849896  385.506248
  388.30935288  391.31046486  417.59194493  441.25158382  442.57636499
  445.47757983  462.08413982  483.7858119   493.98008394  494.6470499
  535.4518199   538.81393504  542.51872182  550.91717696  552.07649183
  585.12791991  631.11332083  637.00035596  662.61578488  691.551471
  694.20643592  700.88655901  701.37186098  701.51687884  701.71605682
  713.74271202  722.75631285  731.85835886  734.36352086  750.48782897
  759.7263      760.56338     764.52853489  768.83656001  786.24181604
  790.67465281  807.08227086  828.01881385  837.05633903  838.45789599
  841.60343289  851.05533791  865.79466891  878.03645492  891.9328289
  892.12592793  895.51519704  908.988518    912.30326986  919.31714797
  919.74718189

In [22]:
###############################################################
####### GENERATE LONGITUDINAL SVM CURVES (VIRIDIS PLOTS) ######
###############################################################

# 6 panels with SVM curves in vidiris; ALREADY GEENERATED
names = ['IA1','IA2','IA3','IJ1','IJ2','AQ2']

# 
fig, ax = plt.subplots()
import scipy
from statsmodels.stats.multitest import multipletests

# 
smooth = True
smooth_window = 30
significance = 0.05
auc = []
early = []
from tqdm import tqdm, trange
for k, name in enumerate(names):
    fnames = np.loadtxt('/media/cat/4TBSSD/yuki/'+name+'/concatenated_svm.txt',dtype='str') #[::-1]

    ax=plt.subplot(2,3,k+1)
    colors = plt.cm.viridis(np.linspace(0,1,len(fnames)))

    auc.append([])
    early.append([])
    for ctr,fname in tqdm(enumerate(fnames)):

        data = np.load(fname, allow_pickle=True)
        acc = data['accuracy']

        if smooth:            
            data = []
            for p in range(acc.shape[1]):
                box = np.ones(smooth_window)/smooth_window
                trace_smooth = np.convolve(acc[:,p], 
                                           box, 
                                           mode='valid')

                data.append(trace_smooth)
            data = np.array(data)
            acc = np.array(data).copy().T

        # 
        mean = acc.mean(1)

        t = np.arange(mean.shape[0])/30-29
        ax.plot(t,mean,
                linewidth=2,
                color=colors[ctr])
    
        # append area under cruve up to t = -10sec
        #print (mean.shape)
        temp = mean[-300:]  # last 10sec prior to movement
        auc[k].append(temp.sum())
        
        sig, earliest = compute_significance(data.T,
                                             significance)
        early[k].append(earliest)
        
    plt.ylim(0.4,1.0)
    plt.xlim(-15,0)
    plt.plot([-30,0],[0.5,0.5],'--',c='black')

if True:
    plt.savefig('/home/cat/edt_trends.svg',dpi=1200)
    plt.close()


28it [00:08,  3.17it/s]


KeyboardInterrupt: 

In [168]:
#########################################
########### EDT FITS ####################
#########################################


# EDT FITS

colors = ['black','blue','red','green','magenta','pink','cyan']
ids= ["M1","M2","M3","M4","M5","M6",]
fig=plt.figure(figsize=(6,6))
for k in range(len(early)):
#for k in [3]:
    ax=plt.subplot(3,2,k+1)
    temp = np.array(early[k])
    idx =np.where(temp<-20)[0]
    temp[idx]=temp[idx-1]
    t = np.arange(temp.shape[0])/temp.shape[0]
    
    #
    lr = LinearRegression()
    lr.fit(t.reshape(-1, 1), temp.reshape(-1, 1))
    #print (k, temp)
    #print (k,"tsta: ", lr.t, "pval: ", lr.p)


    
    temp2 = np.poly1d(np.polyfit(t, temp, 1))(t)
    plt.plot(t, temp2,
             linewidth=6,
             #label=ids[k]+ " ***** ",
             c='black')

    # 
    corr = scipy.stats.pearsonr(t,temp)
    print ("corrL: ", corr)
    
    plt.scatter(t,
                temp, 
                #label=names[k]+ " "+str(round(corr[0],2))+
                #                        " ("+str("pval: {0:.1}".format(corr[1]))+")",
                s=100,
                #linewidth=4,
                c='grey',
                alpha=.8,
               label = str(round(corr[0],2)))
    #t = np.arange(auc1.shape[0])/(auc1.shape[0]-1)

    plt.xticks([])
    plt.yticks([])
    legend = plt.legend(handlelength=0, fontsize=12)
    
    #plt.legend(fontsize=12)
    plt.ylim(-12,0)

#
    plt.xlim(0,1)

if True:
    plt.savefig('/home/cat/edt_fits.svg', dpi=600)
    plt.close()
else:
    plt.show()


corrL:  (0.7014331655212428, 5.6406315699993825e-08)
corrL:  (-0.542035486329535, 0.0011210149237215162)
corrL:  (-0.3419947074557963, 0.02862744552301875)
corrL:  (0.020071514436453608, 0.9021622977083262)
corrL:  (-0.1292660053033484, 0.43286401776660616)
corrL:  (-0.3602431759059126, 0.0011964553177763325)


In [55]:
#############################################################
### COMPUTE DISTRIBUTION OF FIRST SIG DECODING ALL ANIMALS ##
#############################################################
from tqdm import trange

##########################
####### INITIALIZE #######
##########################
# LEVER PULL
vis = Visualize.Visualize()

# lever-related data
vis.main_dir = data_dir
vis.random_flag = False  # shuffle data to show baseline

# 
vis.window = 15
vis.lockout_window = 10
vis.code = 'left_paw'
vis.lockout = False

vis.pca_var = 0.95
vis.pca_flag = True

#################################################
# GENERATE DATA FOR ACCORDEON PLOTS or EDT CURVES
vis.main_dir = data_dir
vis.random_flag = False  # shuffle data to show baseline

# 
vis.window = 30          # # sec out e.g. -30sec ..+30sec
vis.lockout_window = 10
vis.code = 'code_04'
vis.lockout = False
vis.pca_var = 0.95
vis.pca_flag = True
vis.xvalidation = 10

#

# 
vis.min_trials = 20
vis.xlim=[-15,0]
vis.pickle = False
vis.shift_flag = False
vis.apply_lever_offset = False
vis.dlc_offset_flag = False
#
vis.significance=0.05

#
vis.compute_roi = False

#
vis.regular_flag = False
vis.concatenated_flag = True
vis.ideal_window_flag = False

# 
lockouts = [False]

vis.main_dir = '/media/cat/4TBSSD/yuki/'
vis.imaging_rate = 30

#vis.imaging_rate = 15
#vis.main_dir = '/media/cat/4TBSATA/yuki/'
#vis.animal_ids = ['AI3','AK4','AK5','AR4','BA2']
#vis.animal_ids = ['BA2']


# 
vis.sliding_window = vis.imaging_rate   # # of frames to put into SVM data e.g. -1sec .. 0 == 30 frames
vis.smooth_window = vis.imaging_rate   # this is a window for applying filter to output


vis.animal_ids = ['IA1','IA2','IA3','IJ1','IJ2', 'AQ2']


# compute regular sessions
if vis.regular_flag:
    for animal_id in vis.animal_ids:
        vis.animal_id = animal_id
        vis.compute_first_decoding_time(lockouts)

# compute concatenated sessions
elif vis.concatenated_flag:
    for animal_id in vis.animal_ids:
        vis.animal_id = animal_id
        vis.compute_first_decoding_time_concatenated(lockouts)

# 
elif vis.ideal_window_flag:
    for animal_id in vis.animal_ids:
        vis.animal_id = animal_id
        vis.compute_first_decoding_time(lockouts)
    

ANIMAL:  IA1   SESSION:  IA1pm_Feb1_30Hz
 ... data missing /media/cat/4TBSSD/yuki/IA1/tif_files/IA1pm_Feb1_30Hz/IA1pm_Feb1_30Hz_globalPca_min_trials_concatenated200_code_04_30sec_accuracy.npz
skipping, no data

ANIMAL:  IA1   SESSION:  IA1pm_Feb2_30Hz
ANIMAL:  IA1   SESSION:  IA1pm_Feb3_30Hz
ANIMAL:  IA1   SESSION:  IA1pm_Feb4_30Hz
 ... data missing /media/cat/4TBSSD/yuki/IA1/tif_files/IA1pm_Feb4_30Hz/IA1pm_Feb4_30Hz_globalPca_min_trials_concatenated200_code_04_30sec_accuracy.npz
skipping, no data

ANIMAL:  IA1   SESSION:  IA1pm_Feb5_30Hz
 ... data missing /media/cat/4TBSSD/yuki/IA1/tif_files/IA1pm_Feb5_30Hz/IA1pm_Feb5_30Hz_globalPca_min_trials_concatenated200_code_04_30sec_accuracy.npz
skipping, no data

ANIMAL:  IA1   SESSION:  IA1pm_Feb9_30Hz
ANIMAL:  IA1   SESSION:  IA1pm_Feb10_30Hz
ANIMAL:  IA1   SESSION:  IA1pm_Feb11_30Hz
ANIMAL:  IA1   SESSION:  IA1pm_Feb12_30Hz
 ... data missing /media/cat/4TBSSD/yuki/IA1/tif_files/IA1pm_Feb12_30Hz/IA1pm_Feb12_30Hz_globalPca_min_trials_concaten

ANIMAL:  IA2   SESSION:  IA2pm_Feb29_30Hz
ANIMAL:  IA2   SESSION:  IA2pm_Mar1_30Hz
ANIMAL:  IA2   SESSION:  IA2pm_Mar2_30Hz
ANIMAL:  IA2   SESSION:  IA2pm_Mar3_30Hz
ANIMAL:  IA2   SESSION:  IA2am_Mar4_30Hz
ANIMAL:  IA2   SESSION:  IA2am_Mar7_30Hz
ANIMAL:  IA2   SESSION:  IA2pm_Mar8_30Hz
ANIMAL:  IA2   SESSION:  IA2am_Mar9_30Hz
ANIMAL:  IA2   SESSION:  IA2am_Mar10_30Hz
 ... data missing /media/cat/4TBSSD/yuki/IA2/tif_files/IA2am_Mar10_30Hz/IA2am_Mar10_30Hz_globalPca_min_trials_concatenated200_code_04_30sec_accuracy.npz
skipping, no data

ANIMAL:  IA2   SESSION:  IA2am_Mar11_30Hz
ANIMAL:  IA2   SESSION:  IA2pm_Mar14_30Hz
ANIMAL:  IA2   SESSION:  IA2am_Mar15_30Hz
ANIMAL:  IA2   SESSION:  IA2pm_Mar16_30Hz
ANIMAL:  IA2   SESSION:  IA2pm_Mar17_30Hz
ANIMAL:  IA2   SESSION:  IA2pm_Mar18_30Hz
ANIMAL:  IA2   SESSION:  IA2pm_Mar21_30Hz
ANIMAL:  IA2   SESSION:  IA2pm_Mar23_30Hz
ANIMAL:  IA2   SESSION:  IA2pm_Mar24_30Hz
ANIMAL:  IA2   SESSION:  IA2pm_Mar29_30Hz
 ... data missing /media/cat/4TBSSD/y

ANIMAL:  IJ2   SESSION:  IJ2am_Mar9_30Hz
ANIMAL:  IJ2   SESSION:  IJ2am_Mar10_30Hz
ANIMAL:  IJ2   SESSION:  IJ2am_Mar11_30Hz
ANIMAL:  IJ2   SESSION:  IJ2pm_Mar14_30Hz
ANIMAL:  IJ2   SESSION:  IJ2am_Mar15_30Hz
ANIMAL:  IJ2   SESSION:  IJ2pm_Mar16_30Hz
ANIMAL:  IJ2   SESSION:  IJ2pm_Mar17_30Hz
ANIMAL:  IJ2   SESSION:  IJ2pm_Mar18_30Hz
ANIMAL:  IJ2   SESSION:  IJ2pm_Mar21_30Hz
ANIMAL:  IJ2   SESSION:  IJ2pm_Mar22_30Hz
ANIMAL:  IJ2   SESSION:  IJ2pm_Mar23_30Hz
ANIMAL:  IJ2   SESSION:  IJ2pm_Mar24_30Hz
ANIMAL:  IJ2   SESSION:  IJ2pm_Mar29_30Hz
ANIMAL:  IJ2   SESSION:  IJ2pm_Mar30_30Hz
ANIMAL:  IJ2   SESSION:  IJ2pm_Mar31_30Hz
ANIMAL:  IJ2   SESSION:  IJ2pm_Apr1_30Hz
ANIMAL:  IJ2   SESSION:  IJ2pm_Apr4_30Hz
ANIMAL:  IJ2   SESSION:  IJ2pm_Apr5_30Hz
ANIMAL:  IJ2   SESSION:  IJ2pm_Apr6_30Hz
ANIMAL:  IJ2   SESSION:  IJ2pm_Apr7_30Hz
 # SESSIONS PROCESSED:  39




ANIMAL:  AQ2   SESSION:  AQ2am_Dec9_30Hz
 ... data missing /media/cat/4TBSSD/yuki/AQ2/tif_files/AQ2am_Dec9_30Hz/AQ2am_Dec9_30Hz_globalP

ANIMAL:  AQ2   SESSION:  AQ2am_Jan28_30Hz
ANIMAL:  AQ2   SESSION:  AQ2pm_Jan28_30Hz
ANIMAL:  AQ2   SESSION:  AQ2am_Jan29_30Hz
ANIMAL:  AQ2   SESSION:  AQ2pm_Jan29_30Hz
ANIMAL:  AQ2   SESSION:  AQ2am_Feb2_30Hz
ANIMAL:  AQ2   SESSION:  AQ2am_Feb3_30Hz
ANIMAL:  AQ2   SESSION:  AQ2am_Feb4_30Hz
ANIMAL:  AQ2   SESSION:  AQ2am_Feb5_30Hz
ANIMAL:  AQ2   SESSION:  AQ2am_Feb9_30Hz
ANIMAL:  AQ2   SESSION:  AQ2am_Feb10_30Hz
ANIMAL:  AQ2   SESSION:  AQ2am_Feb11_30Hz
ANIMAL:  AQ2   SESSION:  AQ2am_Feb12_30Hz
ANIMAL:  AQ2   SESSION:  AQ2am_Feb15_30Hz
ANIMAL:  AQ2   SESSION:  AQ2am_Feb16_30Hz
ANIMAL:  AQ2   SESSION:  AQ2am_Feb17_30Hz
ANIMAL:  AQ2   SESSION:  AQ2am_Feb18_30Hz
ANIMAL:  AQ2   SESSION:  AQ2am_Feb19_30Hz
ANIMAL:  AQ2   SESSION:  AQ2am_Feb22_30Hz
ANIMAL:  AQ2   SESSION:  AQ2am_Feb23_30Hz
ANIMAL:  AQ2   SESSION:  AQ2am_Feb25_30Hz
 ... data missing /media/cat/4TBSSD/yuki/AQ2/tif_files/AQ2am_Feb25_30Hz/AQ2am_Feb25_30Hz_globalPca_min_trials_concatenated200_code_04_30sec_accuracy.npz
skipping, no

In [43]:
#############################################################
#### PLOT DISTRIBUTION OF FIRST SIG DECODING ALL ANIMALS ####
#############################################################

# REPLACEMENT OF ACORDEON PLOTS

#plot_edt_distributions()        
import pylab as P


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

def plot_edt_distributions_box_plots(vis):

    #vis = Visualize.Visualize()
    vis.regular_flag = True
    vis.ideal_window_flag = False
    #vis.animal_ids = ['IJ1']
    
    fig = plt.figure(figsize=(20,4))
    legend_size = 22
    
    cons = []
    for ctr, animal_id in enumerate(vis.animal_ids):
        ax=plt.subplot(1,6,ctr+1)
        fnames = []

        # plots for Fig 2 comparing 
        if vis.regular_flag:
            fnames.append(vis.root_dir +animal_id+'/first_decoding_time.npz')
            fnames.append(vis.root_dir +animal_id+'/first_decoding_time_lockout.npz')
            fnames.append('/media/cat/4TBSSD/yuki/'+animal_id+'/first_decoding_time_concatenated.npz')
        elif vis.ideal_window_flag:
            fnames.append('/media/cat/4TBSSD/yuki/'+animal_id+'/first_decoding_time.npz')
            fnames.append('/media/cat/4TBSSD/yuki/'+animal_id+'/first_decoding_time_ideal_window.npz')
            
        # 
        shifts=[+15,+15,0]
        #shifts=[0,0,0]
        linestyles = ['-','-','-']
        titles = ['all','lockout','concatenated']
        pvals = [0.05,0.01,0.001,0.0001,0.00001]
        edts = []
        yvals = []
        plotting = False

        for ctr_type, fname in enumerate(fnames):
            #print (" ", animal_id, fname)
            data = np.load(fname, allow_pickle=True)

            # 
            clr = vis.colors[ctr_type]
            res = vis.plot_first_decoding_time_curves(data,
                                                      titles[ctr_type],
                                                      shifts[ctr_type],
                                                      linestyles[ctr_type],
                                                      clr, 
                                                      plotting)
            

            temp = res[0]
            
            # remove low values; don't use fo rnow
            if False:
                idx = np.where(temp<-0.5)[0]
                temp=temp[idx]

            #    
            edts.append(temp)
            yvals.append(res[1])

        
        my_dict = dict(all = edts[0], 
                       lockout = edts[1],
                       concat = edts[2]
                      )
        
        print ("all median: ", np.median(edts[0]))
        print ("lockout median: ", np.median(edts[1]))
        print ("concatenated median: ", np.median(edts[2]))
        
        # 
        cons.append(edts[2])
        data = pd.DataFrame.from_dict(my_dict, orient='index')
        data = data.transpose()


        ###############################################
        ########## PLOT SCATTER #######################
        ###############################################
        if True:
            clrs_local = ["blue","black","red"]
            for i,d in enumerate(data):
                y = data[d]
                x = np.random.normal(i+1, 0.04, len(y))
                if True:
                    plt.plot(x, y, 
                         mfc =clrs_local[i], 
                         mec='k', 
                         ms=7, 
                         marker="o", 
                         linestyle="None",
                         alpha=.8
                        )
                else:
                    plt.scatter(x, y, 
                           c=clrs_local[i],
                           edgecolor='black',
                           s=200,
                           alpha=.7)

            
        ###############################################
        ########## PLOT  #######################
        ###############################################
        flierprops = dict(marker='o', 
                          #markerfacecolor='g', 
                          markersize=10000,
                          linestyle='none', 
                          markeredgecolor='r')
        
        #
        data.boxplot(showfliers=True,
                    flierprops=flierprops)        #if ctr==0:
        #    plt.legend(fontsize=legend_size)

        temp = np.hstack(yvals)
        ymax = np.max(temp)

        #
        #plt.xlim(0.5, 3.5)
        plt.ylim(-15,0)    
        #plt.xticks([])
        #plt.yticks([])

        #
        if False:
        
            plt.plot([0,3.5], [-3,-3],'--',linewidth=3,c='grey',alpha=.5)
            plt.plot([0,3.5],[-10,-10],'--',linewidth=3,c='grey',alpha=.5)

            #ax2 = ax.twinx()  # instantiate a second axes that shares the same x-axis

            patches = []
            res = stats.ks_2samp(edts[0], edts[1])
            print (res)
            label_ = ''
            for k in range(len(pvals)):
                if res[1]<pvals[k]:
                    label_ = label_ + "*"
                else:
                    break

            patches.append(mpatches.Patch(color='blue', label=label_))

            res = stats.ks_2samp(edts[0], edts[2])
            label_ = ''
            for k in range(len(pvals)):
                if res[1]<pvals[k]:
                    label_ = label_ + "*"
                else:
                    break

            patches.append(mpatches.Patch(color='red', label=label_))

            #print ('')
            
            plt.legend(handles=patches,
                       fontsize=legend_size)

        print ('')
        print ('')
    ############################################################
    ############################################################
    ############################################################
    if False:
        plt.show()
    else:
        plt.savefig('/home/cat/first_scatter.svg')
        plt.close()
        
        
    return cons

#
vis.animal_ids = ['AI3']
vis.animal_ids = ['AI3','AK4','AK5','AR4']#,'BA2']
vis.root_dir = '/media/cat/4TBSATA/yuki/'

vis.root_dir = '/media/cat/4TBSSD/yuki/'
vis.animal_ids = ['IA1','IA2','IA3','IJ1','IJ2','AQ2']
#vis.animal_ids = ['IA1']

cons = plot_edt_distributions_box_plots(vis)


temp data allrescontinuous:  (31,)
temp data allrescontinuous:  (31,)
temp data allrescontinuous:  (53,)
all median:  -2.5333333333333314
lockout median:  -2.1000000000000014
concatenated median:  -4.266666666666666


temp data allrescontinuous:  (31,)
temp data allrescontinuous:  (31,)
temp data allrescontinuous:  (33,)
all median:  -3.0333333333333314
lockout median:  -3.2333333333333343
concatenated median:  -5.066666666666666


temp data allrescontinuous:  (38,)
temp data allrescontinuous:  (38,)
temp data allrescontinuous:  (41,)
all median:  -4.2666666666666675
lockout median:  -3.6999999999999993
concatenated median:  -7.600000000000001


temp data allrescontinuous:  (33,)
temp data allrescontinuous:  (33,)
temp data allrescontinuous:  (40,)
all median:  -3.0333333333333314
lockout median:  -2.7333333333333343
concatenated median:  -5.816666666666666


temp data allrescontinuous:  (36,)
temp data allrescontinuous:  (36,)
temp data allrescontinuous:  (39,)
all median:  -4.15
lock

In [61]:
#

fig=plt.figure(figsize=(5,5))
for k in range(6):
    ax=plt.subplot(3,2,k+1)
    y = cons[k]
    x = np.arange(y.shape[0])/y.shape[0]
    
    t = stats.pearsonr(x, y)
    print (t)
    plt.scatter(x,y, c='black', alpha=.5,
                label = str(np.round(t[0],2)))
    
    
    
    # 
    coef = np.polyfit(x,y,1)

    # 
    poly1d_fn = np.poly1d(coef) 
    yy = poly1d_fn(x)
    plt.plot(x, yy, '--',
             c='black',
            linewidth=3) #'--k'=black dashed line, 'yo' = yellow circle marker
    plt.xticks([])
    plt.yticks([])

    plt.legend(fontsize=20)
    plt.ylim(-15,0)  
    plt.xlim(0,x[-1])
    
if False:
    plt.show()
else:
    plt.savefig('/home/cat/longitudinal_concatead_edts.svg')
    plt.close()
    

(0.6929752904834282, 8.919634511314346e-09)
(-0.5436657271735664, 0.001075523390206582)
(-0.30933252065812217, 0.049067247964969786)
(0.12739733455877772, 0.4334030142208259)
(-0.12926600530334867, 0.43286401776660677)
(-0.3036796222954002, 0.006874281357240373)


In [126]:
############################################################
#### PLOT CORRELATION OF DECODING TIME AND NO. SESSIONS ####
############################################################
from tqdm import trange
import matplotlib.cm as cm

vis.smooth_window = 30
vis.min_trials = 20
vis.animal_id = 'IA1'
vis.session = "Feb1_"
vis.window = 30
vis.xlim=-15

#
fig = plt.figure(figsize=(10,8))
vis.significance=0.05 #1E-2
vis.lockout=False
vis.ylims=[200,400]
vis.cmap=cm.Reds
vis.plot_legend = False
# 
clr='blue'

# TWO OPTIONS: singles or contatnated
#fname = '/media/cat/4TBSSD/yuki/first_decoding_time_minTrials20_all_30sec_backup.npz'
fname = '/media/cat/4TBSSD/yuki/first_decoding_time_minTrials20_lockout_30sec_concatenated_backup.npz'
vis.plot_first_decoding_time_vs_n_trials(clr, fname)

# #
if False:
    plt.show()
else:
    plt.savefig('/home/cat/correaltions_concatenated.svg',dpi=1200)
    plt.close()


IA1 Feb1_ [ -4.2         -6.63333333  -4.36666667  -4.63333333  -4.86666667
 -11.43333333 -10.13333333  -6.66666667  -4.43333333  -5.06666667
  -4.2         -4.16666667  -4.33333333  -4.4         -6.33333333
  -4.7         -3.86666667  -4.3         -3.56666667  -3.9
  -3.36666667  -3.96666667  -3.56666667  -3.73333333  -3.86666667
  -3.63333333  -4.26666667  -3.3         -2.86666667  -3.96666667
  -3.66666667  -4.13333333  -3.36666667  -3.26666667  -2.83333333
  -3.63333333  -3.03333333  -3.          -2.7         -3.
  -2.9         -2.66666667  -3.6         -2.43333333  -3.
  -2.46666667  -2.53333333  -2.5         -2.5         -2.16666667
  -2.          -2.          -2.        ]
all preds:  (53, 1)
IA1 Feb1_ [-4.53333333 -4.33333333 -3.13333333 -2.63333333 -2.66666667 -2.
 -2.8        -2.4        -3.13333333 -3.66666667 -2.         -2.
 -2.         -4.96666667 -4.8        -4.83333333 -4.76666667 -5.2
 -8.63333333 -7.36666667 -7.4        -7.2        -7.8        -6.9
 -6.         -4.7   

In [87]:
###################################################
############# PLOT PREDICTOIN TRENDS ##############
###################################################

# FIG 3I  3J
names = ['IA1','IA2','IA3','IJ1','IJ2','AQ2']

auc = []
fig, ax = plt.subplots()

for k, name in enumerate(names):
    fnames = np.loadtxt('/media/cat/4TBSSD/yuki/'+name+'/concatenated_svm.txt',dtype='str') #[::-1]

    ax=plt.subplot(2,3,k+1)
    colors = plt.cm.viridis(np.linspace(0,1,len(fnames)))

    smooth = True
    smooth_window = 30
    auc.append([])
    for ctr,fname in enumerate(fnames):

        data = np.load(fname, allow_pickle=True)
        acc = data['accuracy']
        #print (acc.shape)

        if smooth:            
            data = []
            for p in range(acc.shape[1]):
                box = np.ones(smooth_window)/smooth_window
                trace_smooth = np.convolve(acc[:,p], 
                                           box, 
                                           mode='valid')

                data.append(trace_smooth)

            acc = np.array(data).copy().T

        # 
        mean = acc.mean(1)

        t = np.arange(mean.shape[0])/30-29
        ax.plot(t,mean,
                 linewidth=2,
                color=colors[ctr])
    
        # append area under cruve up to t = -10sec
        print (mean.shape)
        temp = mean[-300:]  # last 10sec prior to movement
        auc[k].append(temp.sum())

# 
fig2=plt.figure()
for k in range(len(auc)):
    plt.plot(auc[k], label=names[k],
            linewidth=4)
plt.legend(fontsize=20)
plt.show()

(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)

In [88]:
#########################################
########### AUC AND EARLY PLOTS #########
#########################################

# FIg 3J 3I

colors = ['black','blue','red','green','magenta','pink','cyan']
ids= ["M1","M2","M3","M4","M5","M6",]
fig2=plt.figure()
for k in range(len(early)):
    temp = np.array(early[k])
    idx =np.where(temp<-20)[0]
    temp[idx]=temp[idx-1]
    t = np.arange(temp.shape[0])
    
    #
    temp2 = np.poly1d(np.polyfit(t, temp, 1))(t)
    plt.plot(t, temp2,
             linewidth=6,
             label=ids[k]+ " ***** ",

             c=colors[k])

    # 
    corr = scipy.stats.pearsonr(t,temp2)
    print ("corr: ", corr)
    
    
    plt.scatter(t,
                temp, 
                #label=names[k]+ " "+str(round(corr[0],2))+
                #                        " ("+str("pval: {0:.1}".format(corr[1]))+")",
                s=100,
                linewidth=4,
                c=colors[k],
                alpha=.3)
    #t = np.arange(auc1.shape[0])/(auc1.shape[0]-1)


#
plt.xlim(0,80)
plt.ylim(-12,0)
plt.legend(fontsize=20)
plt.show()


corr:  (1.0, 0.0)
corr:  (-0.9999999999999999, 3.3310483861078736e-244)
corr:  (-1.0, 0.0)
corr:  (-1.0, 0.0)
corr:  (-1.0, 0.0)
corr:  (-0.9999999999999999, 0.0)


In [169]:
data = np.load('/media/cat/4TBSSD/yuki/IJ2/tif_files/IJ2pm_Mar2_30Hz/IJ2pm_Mar2_30Hz_whole_stack_trial_ROItimeCourses_15sec_pca30components.npy')
print (data.shape)

(40000, 30)


In [69]:
##############################################
##############################################
##############################################

from scipy.signal import savgol_filter

codes = ['Retrosplenial', 'barrel', 'motor', 'limb', 'visual']
clrs=['black','blue','red','magenta','brown']
smoothing = True
min_trials = 30
    
######################    
root_dir = '/media/cat/4TBSSD/yuki/'
animal_id = 'AQ2'

sessions = Visualize.get_sessions(root_dir,animal_id,'all')

print ("SESSIONS: ", sessions)
ctr_sess=0
fig = plt.figure()
for session_id in sessions:
    fnames = []
    for k in range(len(codes)):
        fnames.append(root_dir+ animal_id+'/SVM_Scores/SVM_Scores_ROI_'+
                      session_id+"_"+
                      codes[k]+'_trial_ROItimeCourses_30sec_Xvalid10_Slidewindow30.npz')

    try:
        temp = np.loadtxt(os.path.join(root_dir,animal_id,'tif_files',session_id,
                              session_id+"_all_locs_selected.txt"), dtype='str')
    except:
        print (" ntrial file missing: ")
        continue
        
    if len(temp)<min_trials:
        print ("too few trials: ", len(temp))
        continue

    ax=plt.subplot(8,10,ctr_sess+1)
    for ctr,fname in enumerate(fnames):
        try:
            data = np.load(fname, allow_pickle=True)
        except:
            print ("SVM not compued yet, skipping")
            continue

        acc_r = data['accuracy']

        if smoothing:
            for k in range(acc_r.shape[1]):
                acc_r[:,k] = savgol_filter(acc_r[:,k], 7, 3)

        #
        mean_r = acc_r[:900].mean(1)
        std_r = np.std(acc_r[:900],axis=1)

        t = np.arange(mean_r.shape[0])/30-29

        plt.plot(t,mean_r, linewidth=3,
                label=codes[ctr])
        if False:
            ax.fill_between(t, mean_r+std_r,
                           mean_r-std_r,
                           color=clrs[ctr],
                           alpha=.2)
    # 
    if ctr_sess==0:
        plt.legend()
    else:
        plt.xticks([])
        plt.yticks([])

    plt.ylabel("# trials: "+str(len(temp)))
    plt.xlim(-5,0)
    plt.ylim(0,1.0)
    plt.plot([-30,0],[0.5,0.5],'--',c='grey')

    ctr_sess+=1
   #except:
   #     pass
    
plt.show()



SESSIONS:  ['AQ2am_Dec9_30Hz' 'AQ2am_Dec10_30Hz' 'AQ2pm_Dec10_30Hz'
 'AQ2am_Dec11_30Hz' 'AQ2pm_Dec14_30Hz' 'AQ2am_Dec14_30Hz'
 'AQ2pm_Dec16_30Hz' 'AQ2am_Dec17_30Hz' 'AQ2pm_Dec17_30Hz'
 'AQ2am_Dec18_30Hz' 'AQ2pm_Dec18_30Hz' 'AQ2am_Dec21_30Hz'
 'AQ2am_Dec22_30Hz' 'AQ2am_Dec23_30Hz' 'AQ2am_Dec28_30Hz'
 'AQ2am_Dec29_30Hz' 'AQ2am_Dec30_30Hz' 'AQ2am_Dec31_30Hz'
 'AQ2am_Jan4_30Hz' 'AQ2am_Jan5_30Hz' 'AQ2am_Jan6_30Hz' 'AQ2am_Jan7_30Hz'
 'AQ2am_Jan8_30Hz' 'AQ2pm_Jan11_30Hz' 'AQ2am_Jan11_30Hz'
 'AQ2pm_Jan12_30Hz' 'AQ2am_Jan12_30Hz' 'AQ2pm_Jan13_30Hz'
 'AQ2am_Jan13_30Hz' 'AQ2pm_Jan14_30Hz' 'AQ2am_Jan14_30Hz'
 'AQ2am_Jan15_30Hz' 'AQ2pm_Jan15_30Hz' 'AQ2pm_Jan18_30Hz'
 'AQ2am_Jan18_30Hz' 'AQ2am_Jan19_30Hz' 'AQ2pm_Jan19_30Hz'
 'AQ2pm_Jan20_30Hz' 'AQ2am_Jan20_30Hz' 'AQ2pm_Jan21_30Hz'
 'AQ2am_Jan21_30Hz' 'AQ2pm_Jan22_30Hz' 'AQ2am_Jan22_30Hz'
 'AQ2am_Jan25_30Hz' 'AQ2pm_Jan25_30Hz' 'AQ2pm_Jan26_30Hz'
 'AQ2am_Jan26_30Hz' 'AQ2am_Jan27_30Hz' 'AQ2pm_Jan27_30Hz'
 'AQ2am_Jan28_30Hz' 'AQ2pm_Jan28_30Hz' 'AQ2am_Ja

<ipython-input-69-81a02e332188>:29: UserWarning: loadtxt: Empty input file: "/media/cat/4TBSSD/yuki/AQ2/tif_files/AQ2am_Dec30_30Hz/AQ2am_Dec30_30Hz_all_locs_selected.txt"
  temp = np.loadtxt(os.path.join(root_dir,animal_id,'tif_files',session_id,


too few trials:  4
too few trials:  27
too few trials:  18
 ntrial file missing: 
too few trials:  2
 ntrial file missing: 
too few trials:  26
 ntrial file missing: 
too few trials:  13
 ntrial file missing: 
 ntrial file missing: 
 ntrial file missing: 
too few trials:  2
 ntrial file missing: 
 ntrial file missing: 
too few trials:  3
too few trials:  4
 ntrial file missing: 
 ntrial file missing: 
 ntrial file missing: 
 ntrial file missing: 
too few trials:  3
too few trials:  21
SVM not compued yet, skipping
SVM not compued yet, skipping
SVM not compued yet, skipping
SVM not compued yet, skipping
SVM not compued yet, skipping
SVM not compued yet, skipping
SVM not compued yet, skipping
SVM not compued yet, skipping
SVM not compued yet, skipping
SVM not compued yet, skipping
too few trials:  0


<ipython-input-69-81a02e332188>:29: UserWarning: loadtxt: Empty input file: "/media/cat/4TBSSD/yuki/AQ2/tif_files/AQ2am_Feb25_30Hz/AQ2am_Feb25_30Hz_all_locs_selected.txt"
  temp = np.loadtxt(os.path.join(root_dir,animal_id,'tif_files',session_id,


SVM not compued yet, skipping
SVM not compued yet, skipping
SVM not compued yet, skipping
SVM not compued yet, skipping
SVM not compued yet, skipping
SVM not compued yet, skipping
SVM not compued yet, skipping
SVM not compued yet, skipping
SVM not compued yet, skipping
SVM not compued yet, skipping
SVM not compued yet, skipping
SVM not compued yet, skipping
SVM not compued yet, skipping
SVM not compued yet, skipping
SVM not compued yet, skipping
SVM not compued yet, skipping
SVM not compued yet, skipping
SVM not compued yet, skipping
SVM not compued yet, skipping
SVM not compued yet, skipping
SVM not compued yet, skipping
SVM not compued yet, skipping
SVM not compued yet, skipping
SVM not compued yet, skipping
SVM not compued yet, skipping


In [13]:
#############################################
############## CHECK PCA STMS ###############
#############################################

data = np.load('/media/cat/4TBSSD/yuki/AI3/tif_files/AI3_Nov1pm_15Hz_8x8/AI3_Nov1pm_15Hz_8x8_code_04_trial_ROItimeCourses_15sec_pca_0.95.npy')
print (data.shape)

t = np.arange(451)/15-15
for k in range(3):
    plt.plot(t,data[:,k].mean(0))


plt.show()

(421, 10) (421, 10)


In [20]:
#############################################
############## CHECK SVM RESULTS ############
#############################################
#data1= np.load('/media/cat/4TBSSD/yuki/AI3/SVM_Scores/SVM_Scores_AI3_Dec3am_15Hz_8x8code_04_trial_ROItimeCourses_15sec_Xvalid10_Slidewindow30.npz',
#               allow_pickle=True)
#fname = '/media/cat/4TBSSD/yuki/AI3/SVM_Scores/SVM_Scores_AI3_Nov12am_15Hz_8x8code_04_trial_ROItimeCourses_15sec_Xvalid10_Slidewindow15.npz'
#fname = '/media/cat/4TBSSD/yuki/AQ2/SVM_Scores/SVM_Scores_ROI_AQ2am_Feb19_30Hz_limb, layer 1 - left_trial_ROItimeCourses_30sec_Xvalid10_Slidewindow30.npz'
#fname = '/media/cat/4TBSSD/yuki/AQ2/SVM_Scores/SVM_Scores_ROI_AQ2am_Feb16_30Hz_limb, layer 1 - left_trial_ROItimeCourses_30sec_Xvalid10_Slidewindow30.npz'
#fname = '/media/cat/4TBSSD/yuki/AQ2/SVM_Scores/SVM_Scores_ROI_AQ2am_Mar3_30Hz_limb, layer 1 - left_trial_ROItimeCourses_30sec_Xvalid10_Slidewindow30.npz'

fname = '/media/cat/4TBSSD/yuki/IJ2/SVM_Scores/SVM_Scores_IJ2pm_Mar3_30Hzcode_04_trial_ROItimeCourses_30sec_Xvalid10_Slidewindow30.npz'



data2 = np.load(fname, allow_pickle=True)
#acc1 = data1['accuracy']
acc2 = data2['accuracy']
print (acc2.shape)

pred = data2['predictions']
print ("# trials: ", pred.shape[1]/2)

fig=plt.figure()
ax=plt.subplot(111)

if '30Hz' in fname:
    t = np.arange(acc2.shape[0])/30.-29
    
else:
    t = np.arange(acc2.shape[0])/15.-14
    #plt.plot(t,acc1.mean(1),c='black')
mean = acc2.mean(1)
std = np.std(acc2,1)
plt.plot(t,mean,c='blue')

ax.fill_between(t, mean+std,
               mean-std,
               color='blue',
               alpha=.2)
plt.xlim(-15,15)
plt.ylim(0.4,1)
plt.plot([0,0],[0,1],'--',c='grey')
plt.plot([-15,15],[0.5,0.5],'--',c='grey')
plt.show()


(1771, 10)
# trials:  48.0


(421, 218)
